<a href="https://colab.research.google.com/github/mottasilvia/UCU-NLP/blob/main/Proyecto%20Final/RAG_con_OpenAI_y_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementación de RAG con OpenAI y LangChain



Este notebook tiene como objetivo construir un sistema de Recuperación de Información Guiada por la Pregunta (RAG) utilizando LangChain, ChromaDB y la API de OpenAI. El sistema permite cargar documentos PDF, procesarlos, dividirlos en fragmentos manejables, crear representaciones vectoriales de estos fragmentos y luego realizar consultas sobre ellos para recuperar información relevante.

Índice del Contenido

    Instalación de Bibliotecas Necesarias
    Configuración del Entorno
    Carga y Procesamiento de Documentos PDF
    División de Documentos en Fragmentos
    Creación de Embeddings y Base de Datos Vectorial con OpenAI
    Consultas en la Base de Datos Vectorial
    Recuperación Contextual y Compresión
    Cadena de QA con Recuperación de Contexto (RAG)
    Implementación de un Prompt Personalizado para el RAG
    Ejecución de Consultas de Prueba y Almacenamiento de Resultados


# Instalación de Bibliotecas Necesarias

In [ ]:
! pip install langchain trulens_eval chromadb openai
!pip install tqdm
!pip install PyMuPDF
!pip install pypdf
!pip install sentence-transformers
!pip install langchain_chroma
!pip install tiktoken



  Using cached PyMuPDF-1.24.7-cp310-none-manylinux2014_x86_64.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Configuración del Entorno

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import fitz  # PyMuPDF
from tqdm import tqdm
import os
from langchain.document_loaders import PyPDFLoader

Carga y Procesamiento de Documentos PDF

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.2 MB/s eta 0:00:00


In [ ]:
loaders = []
for archivo in os.listdir('/content/drive/My Drive/UCU-NLP/RAG'):
  # Add a '/' between the directory path and the filename
  file_path = os.path.join('/content/drive/My Drive/UCU-NLP/RAG', archivo)
  loaders.append(PyPDFLoader(file_path))

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
#To count the number of items in the 'documents' list, use the len() function
num_pdfs = len(loaders)
print(num_pdfs)

13


In [ ]:
#To count the number of items in the 'documents' list, use the len() function
num_documents = len(docs)
print(num_documents)

247


División de Documentos en Fragmentos (Chuncks)

In [ ]:
# Define the Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

#Create a split of the document using the text splitter
splits = text_splitter.split_documents(docs)

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install langchain_chroma

In [ ]:
# Create vectod db with open ai embbedings
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()
persist_directory = 'content/chroma/'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())



1344


Consultas en la Base de Datos Vectorial

In [ ]:
query = "What does the acronym BERT stands for?"
docs = vectordb.similarity_search(query)
print(docs[0].page_content)

docs = vectordb.similarity_search_with_score(query)
print(docs[0][0].page_content, docs[0][1])

docs_marg_relevance = vectordb.max_marginal_relevance_search(query, k=2, fetch_k=3)
print(docs_marg_relevance[0])
print(docs_marg_relevance[1])


tional Institute for Clinical Excellence; 
2003.
140. Steinsbekk A, Rygg L, Lisulo M et al. 
Group based diabetes self-manage -
ment education compared to routine 
treatment for people with type 2 di -
abetes mellitus. A systematic review 
with meta-analysis. BMC Health Serv 
Res 2012;12:213.
141. Ramadasa A, Queka KF, Chana CKY
tional Institute for Clinical Excellence; 
2003.
140. Steinsbekk A, Rygg L, Lisulo M et al. 
Group based diabetes self-manage -
ment education compared to routine 
treatment for people with type 2 di -
abetes mellitus. A systematic review 
with meta-analysis. BMC Health Serv 
Res 2012;12:213.
141. Ramadasa A, Queka KF, Chana CKY 0.5206233859062195
page_content='tional Institute for Clinical Excellence; 
2003.
140. Steinsbekk A, Rygg L, Lisulo M et al. 
Group based diabetes self-manage -
ment education compared to routine 
treatment for people with type 2 di -
abetes mellitus. A systematic review 
with meta-analysis. BMC Health Serv 
Res 2012;12:213.
141. Ramada

Recuperación Contextual y Compresión

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.llms import OpenAI

# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

question = "what did the papers say about self rag?"
compressed_docs = compression_retriever.get_relevant_documents(question)
print(compressed_docs[0].page_content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


115. Christopher P. Cannon, M.D., Michael 
A. Blazing, M.D., Robert P. Giuglia -
no, M.D., et al for the IMPROVE-IT 
Investigators* N Engl J Med 2015; 
372:2387-2397June 18, 2015DOI: 
10.1056/NEJMoa1410489
116. Cannon CP, Blazing MA, Giugliano RP, 
et al. Ezetimibe added to statin ther -
apy after acute coronary syndromes.


Cadena de QA y Ejecución de Consultas

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = OpenAI(temperature=0)
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

# Pass question to the qa_chain
question = "¿Cuándo y cómo se debe iniciar tratamiento con antiagregantes plaquetarios en pacientes con DM2?"
result = qa_chain({"query": question})
result["result"]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' Según los estudios mencionados, se recomienda iniciar tratamiento con ácido acetil salicílico (AAS) o clopidogrel en pacientes con DM2 que presenten riesgo cardiovascular, como prevención secundaria. La dosis y frecuencia de administración dependerá de cada caso en particular y debe ser evaluada por un médico.'

In [ ]:
#question = "Any of the papers mention what BERT acronym means?"
question="¿Cuándo y cómo se debe iniciar tratamiento con antiagregantes plaquetarios en pacientes con DM2?"

result = qa_chain({"query": question})
# Check the result of the query
result["result"]

' Según el contexto proporcionado, el tratamiento con antiagregantes plaquetarios en pacientes con DM2 se debe iniciar en la prevención secundaria, es decir, en aquellos pacientes que ya han sufrido un evento cardiovascular. Se recomienda el uso de ácido acetil salicílico (AAS) o clopidogrel, y la dosis y frecuencia de administración deben ser determinadas por un médico.'

Cadena de QA con Recuperación de Contexto (RAG)

Implementación de un Prompt Personalizado para el RAG

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
RAG_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
   llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": RAG_PROMPT}
)

In [ ]:
# Realizar consultas de prueba
resultado=test_qa('para que puedo utilizar k-means ?')
print(resultado)

Consulta: para que puedo utilizar k-means ?
Respuesta: {'query': 'para que puedo utilizar k-means ?', 'result': ' K-means puede ser utilizado para agrupar a los sujetos con diagnóstico reciente de diabetes en función de seis variables clínicas preestablecidas, lo que permite identificar diferentes fenotipos de diabetes y adaptar estrategias terapéuticas específicas para cada grupo. Gracias por preguntar!', 'source_documents': [Document(metadata={'page': 4, 'source': '/content/drive/My Drive/UCU-NLP/RAG/revistadiabetes_or.pdf'}, page_content='DEBES SABERDiabetes6\nUtilizando registros médicos electróni-\ncos y datos extensos de genes, se han \nbuscado modelos basados en métodos \nde topología, de índices de riesgo genéti-\nco, de análisis trayectoria de parámetros \nclínicos y técnicas de agrupamiento so-\nbre variables clínicas.\nLa última metodología es la que más re-\npercusión clínica ha tenido en base a los \nestudios de Ahlqvist y cols. Con el algo-\nritmo “K-means”, agrupan a los

Ejecución de Consultas de Prueba y Almacenamiento de Resultados

In [ ]:
import pandas as pd

# Lista para almacenar preguntas y respuestas
qa_pairs = []

# Función para realizar una consulta y guardar la respuesta
# Función para realizar una consulta y guardar la respuesta con metadatos
def test_qa(query):
    result = qa_chain({"query": query})
    qa_pairs.append({"Pregunta": query, "Respuesta": result})
    print("Consulta:", query)
    print("Respuesta:", result)
    print()

# Ejemplos de consultas de prueba
test_queries = [
    "¿Cuál es la capital de Francia?",
    "¿Qué es la diabetes gestacional?",
    "¿Necesito tomar insulina durante el resto de mi vida si tengo diabetes tipo 1?",
    "¿Qué es el machine learning?",
    "¿Cómo se trata la diabetes tipo 2?"
    "¿Cuáles son los factores de riesgo para desarrollar diabetes tipo 2?"
    "¿Es necesario monitorear el azúcar en sangre aún cuando me siento bien?"
]
# Realizar consultas de prueba


# Realizar consultas de prueba
for query in test_queries:
    test_qa(query)


# Crear un DataFrame con las preguntas, respuestas y contextos
df = pd.DataFrame(qa_pairs)

# Mostrar el DataFrame
print(df)

# Guardar el DataFrame en un archivo CSV
df.to_csv('/content/drive/MyDrive/UCU-NLP/preguntas_respuestas_langchain_v2.csv', index=False)

Consulta: ¿Cuál es la capital de Francia?
Respuesta: {'query': '¿Cuál es la capital de Francia?', 'result': ' Lo siento, no tengo esa información. Gracias por preguntar.', 'source_documents': [Document(metadata={'page': 31, 'source': '/content/drive/My Drive/UCU-NLP/RAG/navarra_es.pdf'}, page_content='Más ejemplares e información: \nINSTITUTO DE SALUD PÚBLICA\nSección de Promoción de Salud\nTfno. 848 42 34 50\nwww.cfnavarra.es/isp'), Document(metadata={'page': 29, 'source': '/content/drive/My Drive/UCU-NLP/RAG/navarra_es.pdf'}, page_content='Federación de Diabéticos Españoles \nC/ Francisco de Rojas, 9, 1° dcha. 4 \n28010 Madrid \nApartado de Correos 3206 \nTel. 91 447 00 35 - Fax 91 447 00 35\ne-mail: fede@federaciondiabetes.org\nwww.federaciondiabetes.org\nAsociación Navarra de Diabéticos Tipo 1\nC/ Carlos III, 30-1°. Oficina 6 - 31002\nPamplona \nTel. 948 153586\nAsociación de Diabéticos ¨La Ribera”\nCentro Cívico Lestonac\nC/ San MarciaI, 25  - 31500 Tudela \nTel. 948 826 367\nFede